<a href="https://colab.research.google.com/github/ardilapablo/VisualizacionDeDatos/blob/main/CLASE2_VISUALIZACION_DE_DATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd

# Ruta completa al archivo CSV en tu Google Drive
ruta_csv = '/content/drive/My Drive/datos_ejercicio_ventas.csv'

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(ruta_csv)

# Mostrar las primeras filas para verificar que se cargó correctamente
df.head()


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [11]:
import plotly.express as px

# Contar cuántos datos hay de cada tipo en la columna 'SCENARIO'
conteo_scenario = df['SCENARIO'].value_counts().reset_index()

# Renombrar las columnas para que sean más descriptivas
conteo_scenario.columns = ['SCENARIO', 'COUNT']

# Crear el gráfico de barras
fig = px.bar(conteo_scenario, x='SCENARIO', y='COUNT', title='Cantidad de Actuals vs Forecasts',
             labels={'SCENARIO': 'Tipo de Dato', 'COUNT': 'Cantidad'})

# Mostrar el gráfico
fig.show()


HORIZONTE DE PREVISIÓN -
Nº PAISES, PRODUCTOS... -
HISTORICO DE DATOS DE DATOS DE ACTUALS/FORECASTS -
FORECASTS DISTINTOS -

HORIZONTE DE PREVISIÓN